In [1]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
working_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)

In [2]:
# Personnal import 
from utils.utilities_DL import match_period_coverage_with_netmob,get_small_ds
from constants.config import get_args,update_modif
from constants.paths import FOLDER_PATH,FILE_NAME,SAVE_DIRECTORY
from K_fold_validation.K_fold_validation import KFoldSplitter

# Hp Tuning
from HP_tuning.hyperparameter_tuning_ray import HP_tuning


'pynvml' is not available on this environment.


In [3]:
# Load config
model_name = 'STGCN' #'CNN'
dataset_names = ['subway_in','netmob']
args = get_args(model_name,dataset_names)

# Modification : 
args.K_fold = 5
args.ray = True
args.W = 0  # IMPORTANT AVEC NETMOB
args.epochs = 100
args.loss_function_type = 'MSE' # 'quantile'

args = update_modif(args)

# Coverage Period : 
small_ds = False
coverage = match_period_coverage_with_netmob(FILE_NAME,dataset_names=['subway_in','netmob'])
(coverage,args) = get_small_ds(small_ds,coverage,args)

# Choose DataSet and VisionModel if needed: 
dataset_names = ['netmob','subway_in'] # ['calendar','netmob'] #['subway_in','netmob','calendar']
vision_model_name = 'FeatureExtractor_ResNetInspired'  # 'ImageAvgPooling'  #'FeatureExtractor_ResNetInspired' #'MinimalFeatureExtractor',

Model: STGCN, K_fold = 5
!!! Loss function: MSE 


In [ ]:
# Train and Evaluate Model: 
mod_plot = 1 # bokeh plotting every epoch 

# Load K-fold subway-ds 
folds = [0] # Here we use the first fold for HP-tuning. 

# In case we need to compute the Sliding K-fold validation:
# folds = np.arange(1,args.K_fold)

K_fold_splitter = KFoldSplitter(dataset_names,args,coverage,vision_model_name,folds)
K_subway_ds,dic_class2rpz = K_fold_splitter.split_k_fold()

Time-step per hour: 4.0
coverage period: 2019-03-16 00:00:00 - 2019-05-31 23:45:00

Init Subway-In Dataset:  torch.Size([7392, 40])
Number of Nan Value:  tensor(0)
Total Number of Elements:  295680 


U size:  torch.Size([6719, 40, 7]) Utarget size:  torch.Size([6719, 40, 1])
U_train size:  torch.Size([3912, 40, 7]) Utarget_train size:  torch.Size([3912, 40, 1])
U_valid size:  torch.Size([1304, 40, 7]) Utarget_valid size:  torch.Size([1304, 40, 1])
U_test size:  torch.Size([1310, 40, 7]) Utarget_test size:  torch.Size([1310, 40, 1])
U_train min:  tensor(0.) U_train max:  tensor(10798.)
U_valid min:  tensor(0.) U_valid max:  tensor(1405.)
U_test min:  tensor(0.) U_test max:  tensor(1760.)
Init NetMob Dataset:  torch.Size([7392, 40, 6, 22, 22])
Number of Nan Value:  tensor(0)
Total Number of Elements:  858654720 


U size:  torch.Size([6719, 40, 6, 22, 22, 7]) Utarget size:  torch.Size([6719, 40, 6, 22, 22, 1])
U_train size:  torch.Size([3912, 40, 6, 22, 22, 7]) Utarget_train size:  torc

## HP Tuning on Fold0:

In [ ]:
num_samples = 20
subway_ds = K_subway_ds[0]
analysis = HP_tuning(subway_ds,args,args_embedding,args_vision,num_samples,dic_class2rpz,working_dir)

## Classical Training: 

In [5]:
from trainer import Trainer
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from plotting.plotting_bokeh import plot_bokeh

args.mixed_precision = True
#args.torch_compile = True
#args_vision.h_dim = 16
subway_ds = K_subway_ds[0]
args.epochs = 3
# Load Model:
model = load_model(subway_ds,args,dic_class2rpz)

# Load Optimizer, Scheduler, Loss function: 
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)

trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = scheduler,dic_class2rpz = dic_class2rpz,show_figure = True)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
# Train Model 
trainer.train_and_valid(mod = 1000,mod_plot = None)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

pi,pi_cqr = plot_bokeh(trainer,subway_ds.normalizer,subway_ds.tensor_limits_keeper.df_verif_test,args)

model size: 0.004GB
number of total parameters: 1151659
number of trainable parameters: 1151659


RuntimeError: Python 3.11+ not yet supported for torch.compile